In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as stats
import argparse
import warnings
warnings.filterwarnings("ignore")
import csv

In [2]:
gene_name = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/ensg_gene_name.csv',sep='\t').drop_duplicates()

In [5]:
path = '/data1/HOMO_PANGENOME/DYY/data/tumor_spec_trans_protein_types_by_tissue/'
files = os.listdir(path)
tissue = [x[:3] for x in files if x[-3:]=='csv']
# files.remove('.ipynb_checkpoints')
# convert_df = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/mstrg_ensg_transcript.csv',sep='\t')
convert = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/bambu/CPM_transcript.txt',sep='\t')[['TXNAME','GENEID']]
gene_df = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/bambu/ref/gencode.v42.annotation.gtf',sep='\t',skiprows=5,header=None)
gene_df_trans = gene_df[gene_df[2]=='transcript']
# # gene_name = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/ensg_gene_name.csv',sep='\t')
# gene_name = gene_name.drop_duplicates()
# convert = convert_df.merge(gene_name,left_on='ensg',right_on='1',how='left').drop('1',axis=1).drop_duplicates('ensg')
convert = convert.merge(gene_name,left_on='GENEID',right_on='1',how='left').drop('1',axis=1).drop_duplicates('GENEID')
uniprot_data = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/uniprot_database/uniprotkb_Human_AND_reviewed_true_AND_m.tsv',sep='\t')
gene_df_8 = gene_df_trans[8].str.split(';',expand=True)
dropna_uniprot = uniprot_data[~uniprot_data.Ensembl.isnull()]

In [6]:
# gene_name = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/normal_chrom_ensg_gene_name.csv',sep='\t')

In [7]:
# grade_df = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/new_grade_df_11.22.csv',sep='\t').rename(columns={'Unnamed: 0':'ensg'})
# grade_df = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/new_grade_df_0709.csv',sep='\t').rename(columns={'Unnamed: 0':'ensg'})
grade_df = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/bambu/s14.csv',sep='\t')
grade_df = grade_df[grade_df['score']>0]

In [8]:
ensg_list = grade_df.ensg.tolist()

In [9]:
len(ensg_list)

11692

In [20]:
ensg_list = pd.read_csv('/data1/HOMO_PANGENOME/DYY/data/bambu/123.txt',header=None)

In [21]:
ensg_list = ensg_list[0].tolist()

In [23]:
function_df = pd.DataFrame(columns=enst_df.columns)
# not_found_df = []
for g in ensg_list:
    ensg_df = gene_df_8[gene_df_8[0].str.contains(g)]
    enst_list = ensg_df[1].str.split('"',expand=True).iloc[:,1].tolist()
    enst_df = pd.DataFrame()
    # score = grade_df.loc[grade_df.ensg==g,'0'].tolist()[0]
    score = grade_df.loc[grade_df.ensg==g,'score'].tolist()[0]
    for l in enst_list:
        try:
            enst_df = pd.concat([enst_df,dropna_uniprot[dropna_uniprot['Ensembl'].str.contains(l)]])
        except:
            continue
    enst_df = enst_df.drop_duplicates()
    if len(enst_df)>0:
        enst_df.loc[:,'ensg'] = g

        # if len(enst_df)==0:
        #     break
        enst_df.loc[:,'score'] = score
        function_df = pd.concat([function_df,enst_df])
    # break

In [28]:
enst_df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Involvement in disease,Function [CC],Pathway,...,Induction,Tissue specificity,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Mutagenesis,Disruption phenotype,Ensembl,ensg,score
17533,P62306,reviewed,RUXF_HUMAN,Small nuclear ribonucleoprotein F (snRNP-F) (S...,SNRPF PBSCF,Homo sapiens (Human),86,NaN,FUNCTION: Plays a role in pre-mRNA splicing as...,NaN,...,NaN,NaN,7-methylguanosine cap hypermethylation [GO:003...,catalytic step 2 spliceosome [GO:0071013]; cyt...,RNA binding [GO:0003723],NaN,NaN,ENST00000266735.10;,ENSG00000139343.11,0.6


In [29]:
function_df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Involvement in disease,Function [CC],Pathway,...,Induction,Tissue specificity,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Mutagenesis,Disruption phenotype,Ensembl,ensg,score
3443,Q9H2S6,reviewed,TNMD_HUMAN,Tenomodulin (TeM) (hTeM) (Chondromodulin-1-lik...,TNMD CHM1L UNQ771/PRO1565,Homo sapiens (Human),317,NaN,FUNCTION: May be an angiogenesis inhibitor.,NaN,...,NaN,TISSUE SPECIFICITY: Highly expressed in hypova...,endothelial cell morphogenesis [GO:0001886]; n...,cytoplasm [GO:0005737]; membrane [GO:0016020];...,NaN,NaN,NaN,ENST00000373031.5 [Q9H2S6-1];,ENSG00000000005.6,0.600000
13516,P08603,reviewed,CFAH_HUMAN,Complement factor H (H factor 1),CFH HF HF1 HF2,Homo sapiens (Human),1231,DISEASE: Basal laminar drusen (BLD) [MIM:12670...,FUNCTION: Glycoprotein that plays an essential...,NaN,...,NaN,TISSUE SPECIFICITY: Expressed in the retinal p...,complement activation [GO:0006956]; complement...,blood microparticle [GO:0072562]; extracellula...,complement component C3b binding [GO:0001851];...,"MUTAGEN 337; /note=""H->A: About 10% loss of he...",NaN,ENST00000367429.9;,ENSG00000000971.17,2.086806
4363,Q5TH74,reviewed,STPG1_HUMAN,O(6)-methylguanine-induced apoptosis 2 (MAPO2)...,STPG1 C1orf201,Homo sapiens (Human),334,NaN,FUNCTION: May positively contribute to the ind...,NaN,...,NaN,NaN,positive regulation of apoptotic process [GO:0...,mitochondrion [GO:0005739]; nucleus [GO:0005634],NaN,NaN,NaN,ENST00000003583.12 [Q5TH74-3];ENST00000337248....,ENSG00000001460.18,0.323170
9793,Q6P499,reviewed,NPAL3_HUMAN,NIPA-like protein 3,NIPAL3 NPAL3,Homo sapiens (Human),406,NaN,NaN,NaN,...,NaN,NaN,magnesium ion transport [GO:0015693],membrane [GO:0016020],magnesium ion transmembrane transporter activi...,NaN,NaN,ENST00000003912.7 [Q6P499-2];ENST00000358028.8...,ENSG00000001461.17,0.041071
7770,Q16850,reviewed,CP51A_HUMAN,Lanosterol 14-alpha demethylase (LDM) (EC 1.14...,CYP51A1 CYP51,Homo sapiens (Human),509,NaN,FUNCTION: Sterol 14alpha-demethylase that play...,PATHWAY: Steroid biosynthesis; zymosterol bios...,...,NaN,TISSUE SPECIFICITY: Ubiquitously expressed wit...,cholesterol biosynthetic process [GO:0006695];...,endoplasmic reticulum [GO:0005783]; endoplasmi...,heme binding [GO:0020037]; iron ion binding [G...,NaN,NaN,ENST00000003100.13 [Q16850-1];ENST00000450723....,ENSG00000001630.18,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,Q15417,reviewed,CNN3_HUMAN,"Calponin-3 (Calponin, acidic isoform)",CNN3,Homo sapiens (Human),329,NaN,FUNCTION: Thin filament-associated protein tha...,NaN,...,NaN,TISSUE SPECIFICITY: Expressed in both non-smoo...,actin filament organization [GO:0007015]; acto...,actin cytoskeleton [GO:0015629]; adherens junc...,actin filament binding [GO:0051015]; cadherin ...,NaN,NaN,ENST00000370206.9 [Q15417-1];ENST00000394202.8...,ENSG00000117519.16,0.600000
1640,Q00688,reviewed,FKBP3_HUMAN,Peptidyl-prolyl cis-trans isomerase FKBP3 (PPI...,FKBP3 FKBP25,Homo sapiens (Human),224,NaN,FUNCTION: FK506- and rapamycin-binding protein...,NaN,...,NaN,NaN,NaN,nucleus [GO:0005634],FK506 binding [GO:0005528]; peptidyl-prolyl ci...,NaN,NaN,ENST00000216330.7;ENST00000396062.4;,ENSG00000100442.11,0.600000
2514,Q8N5B7,reviewed,CERS5_HUMAN,Ceramide synthase 5 (CerS5) (LAG1 longevity as...,CERS5 LASS5,Homo sapiens (Human),392,NaN,FUNCTION: Ceramide synthase that catalyzes the...,PATHWAY: Lipid metabolism; sphingolipid metabo...,...,NaN,NaN,ceramide biosynthetic process [GO:0046513]; sp...,endoplasmic reticulum membrane [GO:0005789],DNA binding [GO:0003677]; sphingosine N-acyltr...,"MUTAGEN 26; /note=""N->Q: Reduced N-glycosylati...",NaN,ENST00000317551.12 [Q8N5B7-1];ENST00000422340....,ENSG00000139624.14,0.600000
2646,Q8TBE9,reviewed,NANP_HUMAN,N-acylneuraminate-9-phosphatase (EC 3.1.3.29) ...,NANP C20orf147 HDHD4,Homo sapiens (Human),248,NaN,NaN,PATHWAY: Amino-sug

In [24]:
function_df.drop_duplicates('ensg').set_index(['ensg','score']).to_csv('/data1/HOMO_PANGENOME/DYY/data/bambu/s14_sup.csv',sep='\t')

In [ ]:
function_df[function_df.score>0].drop_duplicates('ensg').set_index(['ensg','score']).to_csv('/data1/HOMO_PANGENOME/DYY/data/bambu/s14_sup.csv',sep='\t')